In [1]:
import pathlib
import shutil

import pandas as pd
import yaml

import dlsproc.extend
import dlsproc.hier

So that *all* the columns of a `pd.DataFrame` are shown:

In [2]:
pd.set_option('display.max_columns', None)

# Directories

The directory containing downloaded data:

In [3]:
data_directory = pathlib.Path.cwd().parent / 'data' / 'agregados'
assert data_directory.exists()
print(data_directory)

/home/manu/dlsproc/data/agregados


A `list` with all the *zip* files that are to be processed

In [4]:
new_zip_files = sorted(data_directory.glob('PlataformasAgregadasSinMenores_2022*.zip'))
new_zip_files

[PosixPath('/home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202201.zip'),
 PosixPath('/home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202202.zip'),
 PosixPath('/home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202203.zip'),
 PosixPath('/home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202204.zip'),
 PosixPath('/home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202205.zip'),
 PosixPath('/home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202206.zip')]

The final output file

In [5]:
output_file = data_directory / 'agregados_sin_menores_multiindex.parquet'
output_file

PosixPath('/home/manu/dlsproc/data/agregados/agregados_sin_menores_multiindex.parquet')

The historical file (in *parquet* format)

In [6]:
# history_file = pathlib.Path.cwd().parent / 'make' / '2018-2021.parquet'
history_file = data_directory / '2018-2021.parquet'
assert history_file.exists()
print(f'{history_file=}')

history_file=PosixPath('/home/manu/dlsproc/data/agregados/2018-2021.parquet')


# Files

The output file is in the beginning a copy of the *history file*

In [7]:
shutil.copy(history_file, output_file)

PosixPath('/home/manu/dlsproc/data/agregados/agregados_sin_menores_multiindex.parquet')

In [8]:
for i, zip_file in enumerate(new_zip_files):
    print(f'appending {zip_file}')
    dlsproc.extend.parquet_with_zip(output_file, zip_file, output_file)

appending /home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202201.zip
appending /home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202202.zip
appending /home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202203.zip
appending /home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202204.zip
appending /home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202205.zip
appending /home/manu/dlsproc/data/agregados/PlataformasAgregadasSinMenores_202206.zip


In [9]:
%ls {output_file.parent}

2018-2021.parquet
2018-2021.pickle
agregados_sin_menores_multiindex.parquet
PlataformasAgregadasSinMenores_2018.zip
PlataformasAgregadasSinMenores_2019.zip
PlataformasAgregadasSinMenores_2020.zip
PlataformasAgregadasSinMenores_2021.zip
PlataformasAgregadasSinMenores_202201.zip
PlataformasAgregadasSinMenores_202202.zip
PlataformasAgregadasSinMenores_202203.zip
PlataformasAgregadasSinMenores_202204.zip
PlataformasAgregadasSinMenores_202205.zip
PlataformasAgregadasSinMenores_202206.zip
tmp/


In [10]:
res_df = pd.read_parquet(output_file)
res_df.shape

(207088, 43)

An Excel file mapping *human-readable* fields to *multiindexed* columns. The latter was processed in `naming.ipynb` to get the file below.

In [15]:
data_scheme_file = pathlib.Path.cwd().parent / 'samples' / 'PLACE.yaml'
assert data_scheme_file.exists()

It provides (as a `dict`) a mapping from *human-readable* names to (maybe nested) fields in *Atom* files

In [16]:
with open(data_scheme_file) as yaml_data:
    data_scheme = yaml.load(yaml_data, Loader=yaml.FullLoader)

In [17]:
flattened_res_df = dlsproc.hier.flatten_columns_names(res_df, data_scheme)
flattened_res_df.head(2)

id  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                   452    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                   452    Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                      title  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                   452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                                                  updated  \
file name                                          entry                                    
PlataformasAgregadasSinMenores_20180217_180137_... 453   2018-01-02 08:01:52.024000+00:00   
                                                   452   2018-01-02 08:02:24.833000+00:00   

                                                         Número de Expediente  \
file name                                          entry                        
PlataformasAgregadasSinMenores_20180217_180137_... 453                1284/17   
                                                   452                1282/17   

                                                         Estado  \
file name                                          entry          
PlataformasAgregadasSinMenores_20180217_180137_... 453      RES   
                                                   452      RES   

                                                                                       Nombre  \
file name                                          entry                                        
PlataformasAgregadasSinMenores_20180217_180137_... 453    Diputación Provincial de Valladolid   
                                                   452    Diputación Provincial de Valladolid   

                                                                                        Objeto del Contrato  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                   452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                          Tipo de Contrato  \
file name                                          entry                     
PlataformasAgregadasSinMenores_20180217_180137_... 453                 3.0   
                                                   452                 3.0   

                                                          Valor estimado del contrato  \
file name                                          entry                                
PlataformasAgregadasSinMenores_20180217_180137_... 453                       89917.95   
                                                   452                      175708.46   

                                                          Presupuesto base sin impuestos  \
file name                                          entry                                   
PlataformasAgregadasSinMenores_20180217_180137_... 453                          89917.95   
                       

In [20]:
flattened_res_df.to_parquet(output_file.with_stem('agregados_sin_menores_multiindex_plano'))